In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.colheader_justify', 'center')
pd.options.display.max_colwidth = 100

In [2]:
from libLiam.data_structures import *
from libLiam.preprocessing import *
from libLiam.feature_processing import *

# utils

In [3]:
def printFormatdfCxtFToRCFT(df, title):
    print("FormalContext " + title)
    print("||")
    for index, row in df.iterrows():
        print("|" + str(row[0]) + "|")
    print()


def printFormatdfCxtRToRCFT(df, title, source, target):
    print("RelationalContext " + title)
    columns = list(df.columns.values)
    print("source " + source)
    print("target " + target)
    print("scaling exist")
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    print(strCol)
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        print(strtmp)
    print()


def retFormatdfCxtFToRCFT(df, title):
    strRet = ""
    strRet += ("FormalContext " + title) + "\n"
    strRet += ("||") + "\n"
    for index, row in df.iterrows():
        strRet += ("|" + str(row[0]) + "|") + "\n"
    strRet += "\n"
    return strRet


def retFormatdfCxtRToRCFT(df, title, source, target):
    strRet = ""
    strRet += ("RelationalContext " + title) + "\n"
    columns = list(df.columns.values)
    strRet += ("source " + source) + "\n"
    strRet += ("target " + target) + "\n"
    strRet += ("scaling exist") + "\n"
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    strRet += (strCol) + "\n"
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        strRet += (strtmp) + "\n"
    strRet += "\n"
    return strRet


def retFormatdfCxtMatrixed(df, title):
    strRet = ""
    strRet += ("FormalContext " + title) + "\n"
    columns = list(df.columns.values)
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    strRet += (strCol) + "\n"
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        strRet += (strtmp) + "\n"
    strRet += "\n"
    return strRet


def union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list


# data is a dataframe of array
def ctxRelPerCtxForms(df_ctxFormX, data):

    tabRet = []
    # On parcours toutes les lignes des données
    for indexData, rowData in data.iterrows():
        tabContains = []
        # On parcours toutes les lignes du contexte formel
        for indexCtx, rowCtx in df_ctxFormX.iterrows():
            # On compare si l'élément est présent
            if rowCtx[0] in rowData[0]:
                tabContains.append("x")
            else:
                tabContains.append("")
        tabRet.append(tabContains)
    return tabRet

## Produits

In [ ]:
pd.read_csv("CSVs/ctxF_produits.csv").drop("Unnamed: 0", axis=1)

In [14]:
def uniformRoles(roles):
    # Uniformisation des roles
    roles = list(map(lambda x: x.replace('admin', 'administrateur'), roles))
    roles = list(
        map(lambda x: x.replace('administrateuristrateur', 'administrateur'),
            roles))
    roles = list(map(lambda x: x.replace('Presto', ''), roles))
    roles = list(map(lambda x: x.replace('PinkLady', ''), roles))
    roles = list(map(lambda x: x.replace('producteurdepomme', 'producteur'),
                     roles))
    roles = list(map(lambda x: x.replace('utilisateurde', 'utilisateur'), roles))
    roles = list(map(lambda x: x.replace('testeurde', 'testeur'), roles))
    roles = list(
        map(lambda x: x.replace('utilisateurVigneouAmandes', 'utilisateur'),
            roles))
    roles = list(
        map(lambda x: x.replace('utilisateurAmandes', 'utilisateur'), roles))
    roles = list(
        map(lambda x: x.replace('utilisateurVigneouAmandes', 'utilisateur'),
            roles))
    roles = list(
        map(lambda x: x.replace('utilisateurCropwin//Banana', 'utilisateur'),
            roles))
    roles = list(
        map(lambda x: x.replace('responsableCercoban', 'responsable'), roles))
    roles = list(
        map(
            lambda x: x.replace(
                'administrateurjepeuxaffecter(désaffecter)unesaisonàunutilisateur.',
                'administrateur'), roles))
    roles = list(
        map(
            lambda x: x.replace(
                "administrateurjepeuxgérerlessaisonsdel'application(CRUD)",
                'administrateur'), roles))

    # Simplification des roles
    for role in roles:
        if ("administrateur" in role):
            rolesUndupli = list(
                map(lambda x: x.replace(role, 'administrateur'), roles))
        if ("utilisateur" in role):
            rolesUndupli = list(
                map(lambda x: x.replace(role, 'utilisateur'), roles))
        if ("technicien" in role):
            rolesUndupli = list(map(lambda x: x.replace(role, 'technicien'),
                                    roles))
    return rolesUndupli

In [15]:
## garder les US formatés
IDitemToRemove = []
df_completCopy = df_complet.copy(True)
for index, row in df_completCopy.iterrows():
    if row["US_titleFR"][:3] not in ["En ", "en "]:
        IDitemToRemove.append(index)
df_completCopy.drop(IDitemToRemove, axis=0, inplace=True)
df_completCopy["US_titleFR"]

## Séparation des US
roles = []
features = []
for titre in df_completCopy["US_titleFR"]:
    role = titre.split(",")[0][11:].replace(" ", "")
    action = ",".join(titre.split(",")[0:])
    roles.append(role)
    features.append(action)

    
    
roles = uniformRoles(roles)

# Clusturing

In [27]:
def remplacingRole(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace("As ", "")
    sentence = sentence.replace("as ", "")
    sentence = sentence.replace("a ", "")
    sentence = sentence.replace("an ", "")
    sentence = sentence.replace("( ", "")
    sentence = sentence.replace(") ", "")
    sentence = sentence.replace("'", "")
    sentence = sentence.replace(",", "")
    sentence = sentence.replace(" admin", "administrator")
    sentence = sentence.replace(" ", "")
    return sentence

def remplacingFeature(sentence):
    sentence = sentence.replace("'", "")
    sentence = sentence.replace(",", "")
    sentence = sentence.replace("(", " ")
    sentence = sentence.replace(")", " ")
    sentence = sentence.replace("  ", " ")
    return sentence

In [28]:
f = open("data/ITKdatasetEN.txt", "r", encoding="UTF-8")
data = f.read()
f.close()
documents = Documents()
data = data.split('\n')

In [29]:
roleData = []
featureData = []
for us in data:
    usSplit = us.split("I")
    roleData.append(remplacingRole(usSplit[0]))
    featureData.append(remplacingFeature(("".join(usSplit[1:]))[1:]))
del roleData[-1]
roleData = uniformRoles(roleData)
del featureData[-1]
dfRoleFeatureSuperF = pd.DataFrame(data={"Role": roleData})

In [30]:
try:
    documents = Documents()
    data = featureData
    try:
        while(True):
            data.remove('')
    except ValueError:
        pass
    for d in data:
        documents.add_document(Document(d))
except:
    print("ERROR - ReadFromFileError ("+filename+")")

In [31]:
dataset = Dataset()
preprocessing = Preprocessing()
tagged_documents = preprocessing.prepare_parts_of_user_stories(
    documents, False, False, True, False, True, True, "en")

Loading model...
Processing synsets...
[====================]


In [32]:
featureDataList = []
for doc in tagged_documents:
    featureDataList.append(" ".join(doc[0]))
dfRoleFeatureSuperF["Feature"] = featureDataList

In [33]:
featureProcessing = FeatureProcessing()
# dict = map id mot
# doc_term_matrices = idMot par document
dictionary, doc_term_matrices = featureProcessing.generate_dict_dtm(documents)

In [34]:
# get initial features
#initial_features = featureProcessing.initial_features(documents)
#print(type(initial_features[0]))

initial_features = []
for doc in documents:
    initial_features.append(Feature(doc))

In [35]:
# tranform into following lists (feature names, feature keywords, feature tags)
feature_list = []
for feature, tokens, tags, dtm in zip(initial_features,
                                      documents.tokenized_documents,
                                      tagged_documents, doc_term_matrices):
    feature_list.append((feature, tokens, tags, dtm))
grouped_lists = (initial_features, documents.tokenized_documents,
                 tagged_documents, doc_term_matrices)

# grouped_lists = initial_features, documents.tokenized_documents, tagged_documents, doc_term_matrices

In [36]:
vectors = featureProcessing.get_vectors_doc2vec(grouped_lists[2],
                                                grouped_lists[1])

c = Cluster()
num_clusters = c.get_optimal_elbow_num(vectors, 2, len(vectors), 1)
num_clusters

13

In [37]:
num_repeats = 200
max_iter = 2000
clusterer = Cluster()
vectors = featureProcessing.get_vectors_doc2vec(grouped_lists[2],
                                                grouped_lists[1])
clusterer.skl_kmeans(num_clusters, num_repeats, max_iter)
clusters = clusterer.skl_kmeans_fit_and_cluster_vectors(vectors)
sorted_clusters = clusterer._sort_features(clusters, feature_list,
                                           num_clusters)

In [38]:
print(len(sorted_clusters))
count = 0
for i in sorted_clusters:
    count += 1
    print("CLUSTER ", count)
    for u in i:
        #print(u,"\n",type(u[0]), type(u[1]),type(u[2]),type(u[3]),"\n")
        print(" ".join(u[2][0]))
    print()

13
CLUSTER  1
can associate seasons with an account
can specify the characteristics of the irrigation systems of my plot in the user settings
can activate a new season for my plot
can add edit a banana plot
can manually correct the rainfall
can indicate when there is a job change
can login to the application
can modify the display period of my graphs zoom
can change my settings language units
can take notes observations on my plot
can visualize the evolution of nfr and nff
can edit the parameters of a plot current season
visualize the evolution of the indicators of my plot on graphs
visualize the pre climacteric phase on the interface
can visualize the effectiveness of the protections of the plots that manage
can identify my banana trees on my plot
find the network
can remove a phytosanitary treatment

CLUSTER  2
can upload a csv file with my irrigations and my potential measurements
can create weather stations ws available for my farms creation 2 2
must validate the t cs during my fir

In [39]:
dfRoleFeatureSuperF["superF"] = ""

count = -1
for i in sorted_clusters:
    count += 1
    print("CLUSER ", count)
    for u in i:
        idValue = dfRoleFeatureSuperF.index[dfRoleFeatureSuperF['Feature'] == " ".join(u[2][0])].tolist()
        dfRoleFeatureSuperF.at[idValue, "superF"] = str(count)
        

CLUSER  0
CLUSER  1
CLUSER  2
CLUSER  3
CLUSER  4
CLUSER  5
CLUSER  6
CLUSER  7
CLUSER  8
CLUSER  9
CLUSER  10
CLUSER  11
CLUSER  12


In [40]:
dfRoleFeatureSuperF.sort_values(by=['superF'])


,Role,Feature,superF
37,user,can add edit a banana plot,0
118,technician,can remove a phytosanitary treatment,0
115,technician,can identify my banana trees on my plot,0
78,user,can edit the parameters of a plot current season,0
35,user,can activate a new season for my plot,0
...,...,...,...
71,user,can view past seasons of my plots,9
76,user,can view condensed information of my plot on the map,9
86,advisor,can crud holdings and accounts admin,9
44,user,can define my stress zones,9


In [41]:
dfRoleFeatureSuperF

,Role,Feature,superF
0,administrateur,can assign unassign a season to a user,6
1,administrateur,can manage the seasons of the application crud,12
2,administrateur,want to be able to recover data from a plot deleted by the user,8
3,administrateur,have access to the plots of another account,4
4,administrateur,can associate seasons with an account,0
...,...,...,...
122,prestouser,can see my block info on the map pop up,6
123,prestouser,can see the produced productions in the second tab on the right of the detailed view,6
124,prestouser,can see the production forecast on the right tab of the detail view,6
125,user,can enter the weekly productions of my blocks,4


# FCA Multiple

In [42]:
dfRoleFeatureSuperF["Feature"]

0                                                    can assign unassign a season to a user
1                                            can manage the seasons of the application crud
2                           want to be able to recover data from a plot deleted by the user
3                                               have access to the plots of another account
4                                                     can associate seasons with an account
                                               ...                                         
122                                                 can see my block info on the map pop up
123    can see the produced productions in the second tab on the right of the detailed view
124                     can see the production forecast on the right tab of the detail view
125                                           can enter the weekly productions of my blocks
126                                     can see a summary of my info on the bloc

## product2Feature

In [43]:
listproduits = df_complet["products"]
listproduitsperUS = []
for elem in listproduits:
    listTmp = []
    for prod in elem[2:-2].replace("'","").split(" "):
        listTmp.append(prod)
    listproduitsperUS.append(listTmp)

df_produitsPerUS = pd.DataFrame(data={"listproduitsperUS":listproduitsperUS})

In [44]:
df_produitsPerUS

,listproduitsperUS
0,[Cropwin]
1,[Cropwin]
2,[Vintel+Orchards]
3,"[Cropwin, Vintel+Orchards]"
4,[Vintel+Orchards]
...,...
122,[Presto]
123,[Presto]
124,[Presto]
125,[Presto]


In [45]:
#df_product2UFeature["Produits"] = []
dfRoleFeatureSuperF["Produits"] = df_produitsPerUS

In [46]:
product2Feature = my_array = np.array(ctxRelPerCtxForms(df_produits, df_produitsPerUS))

In [47]:
print(len(product2Feature), " ", len(dfRoleFeatureSuperF["Feature"]))

127   127


In [48]:
# df_produits x df_idUS
df_product2UFeature = pd.DataFrame(my_array,
                                    columns=df_produits["Produit"],
                                    index=dfRoleFeatureSuperF["Feature"])
df_product2UFeature.to_csv("CSVs/ctxR_" + "roleFeatureSuperF" + ".csv")
df_product2UFeature

Produit,Presto,Sigatocare,Pink+Lady,Prevent,Cropwin,Vintel+Orchards
Feature,,,,,,
can assign unassign a season to a user,,,,,x,
can manage the seasons of the application crud,,,,,x,
want to be able to recover data from a plot deleted by the user,,,,,,x
have access to the plots of another account,,,,,x,x
can associate seasons with an account,,,,,,x
...,...,...,...,...,...,...
can see my block info on the map pop up,x,,,,,
can see the produced productions in the second tab on the right of the detailed view,x,,,,,
can see the production forecast on the right tab of the detail view,x,,,,,


In [49]:
dfRoleFeatureSuperF

,Role,Feature,superF,Produits
0,administrateur,can assign unassign a season to a user,6,[Cropwin]
1,administrateur,can manage the seasons of the application crud,12,[Cropwin]
2,administrateur,want to be able to recover data from a plot deleted by the user,8,[Vintel+Orchards]
3,administrateur,have access to the plots of another account,4,"[Cropwin, Vintel+Orchards]"
4,administrateur,can associate seasons with an account,0,[Vintel+Orchards]
...,...,...,...,...
122,prestouser,can see my block info on the map pop up,6,[Presto]
123,prestouser,can see the produced productions in the second tab on the right of the detailed view,6,[Presto]
124,prestouser,can see the production forecast on the right tab of the detail view,6,[Presto]
125,user,can enter the weekly productions of my blocks,4,[Presto]


In [50]:
df_product2UFeature.index

Index(['can assign unassign a season to a user',
       'can manage the seasons of the application crud',
       'want to be able to recover data from a plot deleted by the user',
       'have access to the plots of another account',
       'can associate seasons with an account', 'can create an account',
       'can have all the information associated with a block for the current season',
       'can delete an observation',
       'can enter irrigations with different irrigation systems and view them',
       'can specify the characteristics of the irrigation systems of my plot in the user settings',
       ...
       'find the network', 'can remove a phytosanitary treatment',
       'have blocks plots with realistic realized production data for the current season and for past seasons',
       'can create pl admin distributor technician producer accounts',
       'go to a blocks details can see a graph of weekly productions for the current season',
       'can see my block info on the

In [52]:
def toFC(df, column):
    noDuplicate = df[column].drop_duplicates().tolist()
    df_tmp = pd.DataFrame(data={column: list(dict.fromkeys(noDuplicate))})
    df_complet = pd.DataFrame(data={column: df[column].tolist()})
    data = np.array(ctxRelPerCtxForms(df_tmp, df_complet))
    return pd.DataFrame(data, columns=noDuplicate)
    
df_superF = toFC(dfRoleFeatureSuperF, "superF").set_index(df_product2UFeature.index)
df_Role = toFC(dfRoleFeatureSuperF, "Role").set_index(df_product2UFeature.index)
dfcombined = pd.concat([df_Role, df_product2UFeature, df_superF], axis=1)#.drop(['Produits'], axis=1)
dfcombined

,administrateur,analyst,observer,almondsuser,cropwin/presto/bananuser,authenticatedprestouser,vineoralmonduser,prestouser,vintelvigneuser,userwithoutplot,...,4,0,9,11,1,2,5,10,3,7
Feature,,,,,,,,,,,,,,,,,,,,,
can assign unassign a season to a user,x,,,,,,,,,,...,,,,,,,,,,
can manage the seasons of the application crud,x,,,,,,,,,,...,,,,,x,x,,,,
want to be able to recover data from a plot deleted by the user,x,,,,,,,,,,...,,,,,,,,,,
have access to the plots of another account,x,,,,,,,,,,...,x,,,,,,,,,
can associate seasons with an account,x,,,,,,,,,,...,,x,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
can see my block info on the map pop up,,,,,,,,x,,,...,,,,,,,,,,
can see the produced productions in the second tab on the right of the detailed view,,,,,,,,x,,,...,,,,,,,,,,
can see the production forecast on the right tab of the detail view,,,,,,,,x,,,...,,,,,,,,,,


In [ ]:
def getFCPerProduit(df_produits, df_perRole):
    # Product x US
    dfFinal = df_perRole[df_produits["Produit"]].transpose()

    # US x Cluster
    df_usCluster = df_perRole[dict.fromkeys(dfRoleFeatureSuperF["superF"])]
        
    
    # Product x (US + Cluster)
    for newColum in dict.fromkeys(dfRoleFeatureSuperF["superF"]):
        dfFinal[newColum] = ""
            
    # For each product
    for index, row in df_produits.iterrows():
        # Produit
        #print(row["Produit"])
        #print(len(df_perRole.loc[df_perRole[row["Produit"]] == "x"]))
        if(len(df_perRole.loc[df_perRole[row["Produit"]] == "x"]) > 0):
            #print(df_perRole.loc[df_perRole[row["Produit"]] == "x"])
            
            df_perRole.loc[df_perRole[row["Produit"]] == "x"].to_csv("CSVs/ctxF_" + "ProductUSCluster" + ".csv")
        
            # For each cluster
            for cluster in df_perRole[dict.fromkeys(dfRoleFeatureSuperF["superF"])]:
                print(cluster)
            
def getDFProductUS(df_produits, df_perRole):
    # Product x US
    return df_perRole[df_produits["Produit"]].transpose() 

getDFProductUS(df_produits, dfcombined.loc[dfcombined["user"] == "x"])

def getdfProductCluster(df_produits, df_perRole):
    # Product x US
    return df_perRole[df_produits["Produit"]].transpose() 


In [ ]:
for role in list(dict.fromkeys(dfRoleFeatureSuperF["Role"])):
    print("\n",role, len(dfcombined.loc[dfcombined[role] == "x"]))
    getFCPerProduit(df_produits, dfcombined.loc[dfcombined[role] == "x"])

In [ ]:

getDFProductUS(df_produits, dfcombined.loc[dfcombined["user"] == "x"])